In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, Imputer
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats

In [2]:
#Read in Data
train = pd.read_csv('dataFiles/application_train.csv')
test = pd.read_csv('dataFiles/application_test.csv')
bureau_data = pd.read_csv('dataFiles/bureau.csv')
bureau_balance_data = pd.read_csv('dataFiles/bureau_balance.csv')
prev_app_data = pd.read_csv('dataFiles/previous_application.csv')
pos_cash_balance_data = pd.read_csv('dataFiles/POS_CASH_balance.csv')
installments_data = pd.read_csv('dataFiles/installments_payments.csv')
cc_data = pd.read_csv('dataFiles/credit_card_balance.csv')

In [3]:
cc_data_grouped = cc_data.select_dtypes(exclude='object').groupby('SK_ID_PREV', as_index = False).agg({'MONTHS_BALANCE':[sum, 'median', 'mean', min, max],
                                                                       'AMT_BALANCE':[sum, 'median', 'mean', min, max],
                                                                       'AMT_CREDIT_LIMIT_ACTUAL':[sum, 'median', 'mean', min, max],
                                                                       'AMT_DRAWINGS_ATM_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'AMT_DRAWINGS_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'AMT_DRAWINGS_OTHER_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'AMT_DRAWINGS_POS_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'AMT_INST_MIN_REGULARITY':[sum, 'median', 'mean', min, max],
                                                                       'AMT_PAYMENT_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'AMT_PAYMENT_TOTAL_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'AMT_RECEIVABLE_PRINCIPAL':[sum, 'median', 'mean', min, max],
                                                                       'AMT_RECIVABLE':[sum, 'median', 'mean', min, max],
                                                                       'AMT_TOTAL_RECEIVABLE':[sum, 'median', 'mean', min, max],
                                                                       'CNT_DRAWINGS_ATM_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'CNT_DRAWINGS_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'CNT_DRAWINGS_OTHER_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'CNT_DRAWINGS_POS_CURRENT':[sum, 'median', 'mean', min, max],
                                                                       'CNT_INSTALMENT_MATURE_CUM':[sum, 'median', 'mean', min, max],
                                                                       'SK_DPD':[sum, 'median', 'mean', min, max],
                                                                       'SK_DPD_DEF':[sum, 'median', 'mean', min, max]
                                                                      })

In [4]:
cc_data_grouped.columns = ['_CC_'.join(col) if col != ('SK_ID_PREV', '') else col[0] for col in cc_data_grouped.columns]

In [5]:
prev_app_data.columns = [i + '_PRE' if i != 'SK_ID_PREV' and i != 'SK_ID_CURR' else i for i in prev_app_data.columns]

In [6]:
prev_app_data_merged = prev_app_data.merge(cc_data_grouped, how = 'left', left_on = 'SK_ID_PREV', right_on = 'SK_ID_PREV')

In [7]:
installments_data_grouped = installments_data.groupby('SK_ID_PREV', as_index = False).agg({'NUM_INSTALMENT_VERSION':[sum, 'median', 'mean', min, max],
                                                              'NUM_INSTALMENT_NUMBER':[sum, 'median', 'mean', min, max],
                                                              'DAYS_INSTALMENT': [sum, 'median', 'mean', min, max],
                                                              'DAYS_ENTRY_PAYMENT': [sum, 'median', 'mean', min, max],
                                                              'AMT_INSTALMENT': [sum, 'median', 'mean', min, max],
                                                              'AMT_PAYMENT': [sum, 'median', 'mean', min, max]})

In [8]:
installments_data_grouped.columns = ['_INST_'.join(col) if col != ('SK_ID_PREV', '') else col[0] for col in installments_data_grouped.columns]

In [9]:
prev_app_data_merged = prev_app_data_merged.merge(installments_data_grouped, how = 'left', left_on = 'SK_ID_PREV', right_on = 'SK_ID_PREV')

In [10]:
pos_cash_balance_data = pd.get_dummies(pos_cash_balance_data, drop_first = True)

In [11]:
pos_cash_balance_data_grouped = pos_cash_balance_data.groupby('SK_ID_PREV', as_index=False).agg({'MONTHS_BALANCE':[sum, 'median', 'mean', min, max],
                                                                                                 'CNT_INSTALMENT':[sum, 'median', 'mean', min, max],
                                                                                                 'CNT_INSTALMENT_FUTURE':[sum, 'median', 'mean', min, max],
                                                                                                 'CNT_INSTALMENT_FUTURE':[sum, 'median', 'mean', min, max],
                                                                                                 'SK_DPD':[sum, 'median', 'mean', min, max],
                                                                                                 'SK_DPD_DEF':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Amortized debt':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Approved':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Completed':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Returned to the store':[sum, 'median', 'mean', min, max],
                                                                                                 'NAME_CONTRACT_STATUS_Signed':[sum, 'median', 'mean', min, max],
                                                                                                })

In [12]:
pos_cash_balance_data_grouped.columns = ['_POS_'.join(col) if col != ('SK_ID_PREV', '') else col[0] for col in pos_cash_balance_data_grouped.columns]

In [13]:
prev_app_data_merged = prev_app_data_merged.merge(pos_cash_balance_data_grouped, how = 'left', left_on = 'SK_ID_PREV', right_on = 'SK_ID_PREV')

In [14]:
bureau_balance_data_grouped = pd.get_dummies(bureau_balance_data).groupby('SK_ID_BUREAU', as_index=False).agg({'MONTHS_BALANCE':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_0':[sum, 'median', 'mean', min, max],
                                                                                                               'STATUS_1':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_2':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_3':[sum, 'median', 'mean', min, max],
                                                                                                               'STATUS_3':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_4':[sum, 'median', 'mean', min, max], 
                                                                                                               'STATUS_5':[sum, 'median', 'mean', min, max],
                                                                                                               'STATUS_X':[sum, 'median', 'mean', min, max]})

In [15]:
bureau_balance_data_grouped.columns = ['_BBD_'.join(col) if col != ('SK_ID_BUREAU', '') else col[0] for col in bureau_balance_data_grouped.columns]

In [16]:
bureau_data = bureau_data.merge(bureau_balance_data_grouped, how = 'left', left_on = 'SK_ID_BUREAU', right_on = 'SK_ID_BUREAU')

In [17]:
# Joining Bureau Grouped Data with Train Data
bureau_data_grouped = bureau_data.groupby('SK_ID_CURR', as_index=False).agg({'DAYS_CREDIT':[sum, 'median', 'mean', min, max],
                                                                             'CREDIT_DAY_OVERDUE':[sum, 'median', 'mean', min, max],
                                                                             'DAYS_CREDIT_ENDDATE':[sum, 'median', 'mean', min, max],
                                                                             'DAYS_ENDDATE_FACT':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_MAX_OVERDUE':[sum, 'median', 'mean', min, max],
                                                                             'CNT_CREDIT_PROLONG':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_SUM':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_SUM_DEBT':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_SUM_LIMIT':[sum, 'median', 'mean', min, max],
                                                                             'AMT_CREDIT_SUM_OVERDUE':[sum, 'median', 'mean', min, max],
                                                                             'DAYS_CREDIT_UPDATE':[sum, 'median', 'mean', min, max],
                                                                             'AMT_ANNUITY':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_median':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_min':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_X_BBD_max':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_median':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_min':[sum, 'median', 'mean', min, max],
                                                                             'STATUS_5_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'MONTHS_BALANCE_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_4_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_3_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_2_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_1_BBD_max':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_sum':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_median':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_mean':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_min':[sum, 'median', 'mean', min, max],
                                                                            'STATUS_0_BBD_max':[sum, 'median', 'mean', min, max]})

In [19]:
bureau_data_grouped.columns = ['_'.join(col) if col != ('SK_ID_CURR', '') else col[0] for col in bureau_data_grouped.columns]

In [23]:
train_merged = train.merge(bureau_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

test_merged = test.merge(bureau_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

In [24]:
prev_app_data_subset = prev_app_data_merged[prev_app_data_merged.select_dtypes(exclude=['object']).columns.tolist()].drop('SK_ID_PREV', axis = 1)

In [25]:
prev_app_data_grouped = prev_app_data_subset.groupby('SK_ID_CURR', as_index=False).agg([sum,'mean','median','max','min'])

In [26]:
prev_app_data_grouped.columns = ['_'.join(col) if col != ('SK_ID_CURR', '') else col[0] for col in prev_app_data_grouped.columns]

In [27]:
prev_app_data_grouped = prev_app_data_grouped.reset_index()

In [28]:
train_merged = train_merged.merge(prev_app_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

test_merged = test_merged.merge(prev_app_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

In [29]:
own_car_median = train_merged.OWN_CAR_AGE.median()

In [30]:
train_dummies = pd.get_dummies(train_merged[['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']], drop_first= True)

test_dummies = pd.get_dummies(test_merged[['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']], drop_first = True)

In [31]:
def own_car_missing(x):
    if x['FLAG_OWN_CAR'] == 'N':
        return 28
    elif x['FLAG_OWN_CAR'] == 'Y' and pd.isnull(x['OWN_CAR_AGE']):
        return own_car_median
    else:
        return x['OWN_CAR_AGE']

In [32]:
train_merged = pd.concat([train_merged, train_dummies], axis = 1).select_dtypes(exclude=['object'])

test_merged = pd.concat([test_merged, test_dummies], axis = 1).select_dtypes(exclude=['object'])

In [37]:
train_merged_subset = train_merged.dropna(thresh=len(train_merged) - 20000, axis = 1)

In [38]:
new_cols = train_merged_subset.columns.tolist()
new_cols.remove('TARGET')

In [53]:
len(train_merged_subset.columns)

634

In [54]:
col_corr = train_merged_subset.corr()['TARGET'].sort_values()

In [57]:
for i in col_corr.index:
    print i

EXT_SOURCE_2
DAYS_FIRST_DRAWING_PRE_mean
DAYS_EMPLOYED
DAYS_FIRST_DRAWING_PRE_min
RATE_DOWN_PAYMENT_PRE_sum
DAYS_FIRST_DRAWING_PRE_sum
NUM_INSTALMENT_VERSION_INST_median_sum
NUM_INSTALMENT_VERSION_INST_min_min
NUM_INSTALMENT_VERSION_INST_mean_sum
AMT_GOODS_PRICE
CNT_INSTALMENT_POS_max_count
CNT_INSTALMENT_FUTURE_POS_min_count
CNT_INSTALMENT_FUTURE_POS_max_count
CNT_INSTALMENT_POS_median_count
CNT_INSTALMENT_POS_mean_count
CNT_INSTALMENT_FUTURE_POS_mean_count
CNT_INSTALMENT_POS_min_count
CNT_INSTALMENT_FUTURE_POS_median_count
NAME_CONTRACT_STATUS_Completed_POS_sum_count
NAME_CONTRACT_STATUS_Signed_POS_mean_count
NAME_CONTRACT_STATUS_Completed_POS_mean_count
NAME_CONTRACT_STATUS_Completed_POS_min_count
NAME_CONTRACT_STATUS_Completed_POS_median_count
NAME_CONTRACT_STATUS_Amortized debt_POS_median_count
CNT_INSTALMENT_FUTURE_POS_sum_count
NAME_CONTRACT_STATUS_Completed_POS_max_count
NAME_CONTRACT_STATUS_Amortized debt_POS_mean_count
CNT_INSTALMENT_POS_sum_count
NAME_CONTRACT_STATUS_Amortiz

In [ ]:
test_merged_subset = test_merged[new_cols]

In [ ]:
imputer = Imputer(strategy = 'median')
imputer.fit(train_merged_subset.drop('TARGET', axis = 1))
train_merged_imputed = pd.DataFrame(imputer.transform(train_merged_subset.drop('TARGET', axis = 1)), columns = train_merged_subset.drop('TARGET', axis = 1).columns)
test_merged_imputed = pd.DataFrame(imputer.transform(test_merged_subset), columns = train_merged_subset.drop('TARGET', axis = 1).columns)

In [ ]:
train_merged_imputed['DAYS_EMPLOYED_^2'] = train_merged_imputed['DAYS_EMPLOYED'] ** 2
train_merged_imputed['AMT_GOODS_PRICE_^2'] = train_merged_imputed['AMT_GOODS_PRICE'] ** 2
#train_merged_imputed['DAYS_CREDIT_^2'] = train_merged_imputed['DAYS_CREDIT'] ** 2
train_merged_imputed['DAYS_BIRTH_^2'] = train_merged_imputed['DAYS_BIRTH'] ** 2

test_merged_imputed['DAYS_EMPLOYED_^2'] = test_merged_imputed['DAYS_EMPLOYED'] ** 2
test_merged_imputed['AMT_GOODS_PRICE_^2'] = test_merged_imputed['AMT_GOODS_PRICE'] ** 2
#test_merged_imputed['DAYS_CREDIT_^2'] = test_merged_imputed['DAYS_CREDIT'] ** 2
test_merged_imputed['DAYS_BIRTH_^2'] = test_merged_imputed['DAYS_BIRTH'] ** 2

In [ ]:
column_corr = pd.concat([train_merged_imputed, train['TARGET']], axis = 1).corr()['TARGET'].sort_values()

In [ ]:
column_corr_subset = column_corr[(column_corr > .045) | (column_corr < -.025)].index.values.tolist()

In [ ]:
column_corr_subset.remove('TARGET')

In [ ]:
#column_corr = train_merged.corr()['TARGET'].sort_values()
#column_corr = column_corr[(column_corr > .01) | (column_corr < -.01)].index.values.tolist()
#column_corr.remove('TARGET')

In [ ]:
train_subset = train_merged_imputed[column_corr_subset]

test_subset = test_merged_imputed[column_corr_subset]

In [ ]:
poly_transformer = PolynomialFeatures(degree = 2)
poly_transformer.fit(train_subset)
train_poly_features = poly_transformer.transform(train_subset)

In [ ]:
train_subset_poly = pd.DataFrame(train_poly_features, columns = poly_transformer.get_feature_names(
    input_features = train_subset.columns.tolist()
))

In [ ]:
test_poly_features = poly_transformer.transform(test_subset)

In [ ]:
test_subset_poly = pd.DataFrame(test_poly_features, columns = poly_transformer.get_feature_names(input_features = train_subset.columns.tolist()))

In [ ]:
scaler = MinMaxScaler(feature_range = (0, 1))

In [ ]:
scaler.fit(train_subset_poly)
train_scaled = scaler.transform(train_subset_poly)
test_scaled = scaler.transform(test_subset_poly)

In [ ]:
from catboost import CatBoostClassifier
cat_model = CatBoostClassifier(iterations=500, learning_rate=0.1)

In [ ]:
cat_model.fit(train_scaled, train.TARGET)

In [ ]:
test_y_cat = pd.DataFrame(cat_model.predict_proba(test_scaled))
submission_cat = pd.concat([test.SK_ID_CURR, test_y_cat], axis=1).drop(0, axis = 1)
submission_cat.columns = ['SK_ID_CURR', 'Target']

In [ ]:
#max_sub_cat = submission_cat.Target.max()
#min_sub_cat = submission_cat.Target.min()

In [ ]:
#submission_cat['Target'] = submission_cat['Target'].apply(lambda x: (x - min_sub_cat) / (max_sub_cat - min_sub_cat))

In [ ]:
submission_cat.to_csv('reduced_cat17.csv', index = False)

In [ ]:
########################################################################

In [ ]:
column_corr = pd.concat([train_subset_poly, train['TARGET']], axis = 1).corr()['TARGET'].sort_values()
column_corr = column_corr.drop('TARGET')
column_corr = column_corr[(column_corr > .03) | (column_corr < -.03)].index.values.tolist()

In [ ]:
train_subset_poly = train_subset_poly[column_corr]

In [ ]:
test_subset_poly = test_subset_poly[column_corr]

In [ ]:
scaler.fit(train_subset_imputed)
train_scaled = scaler.transform(train_subset_imputed)
test_scaled = scaler.transform(test_subset_imputed)

In [ ]:
cat_model.fit(train_scaled, target)

In [ ]:
test_y_cat = pd.DataFrame(cat_model.predict_proba(test_scaled))
submission_cat = pd.concat([test.SK_ID_CURR, test_y_cat], axis=1).drop(0, axis = 1)
submission_cat.columns = ['SK_ID_CURR', 'Target']
submission_cat.to_csv('reduced_cat11.csv', index = False)